In [3]:
!pip install catboost lightgbm

In [55]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
test = pd.read_csv("test.csv")
df = pd.read_csv("train.csv")
df_new = df.drop("target", axis = 1)

In [56]:
df

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,target,Id
0,8,14,4,1334.0,1440.0,XE,430,N13553,54.0,FAT,SAN,314,3.0,9.0,0,FYNC08ZRT1MS
1,5,3,6,1706.0,2016.0,XE,2567,N12924,103.0,BNA,EWR,748,9.0,18.0,1,4M3EPJ0BFKNV
2,7,11,5,1323.0,1507.0,XE,2989,N19966,125.0,CLT,IAH,913,6.0,33.0,1,OR30D5LJS4AF
3,12,11,4,2039.0,2238.0,WN,3743,N215WN,224.0,BWI,DEN,1491,5.0,10.0,1,0PRTFL3QD5ZI
4,5,23,5,1628.0,1744.0,UA,926,N347UA,60.0,LAX,SFO,337,3.0,13.0,1,DR7UZO31NC2A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,11,22,6,1119.0,1238.0,AA,2421,N383AA,174.0,DFW,LAX,1235,10.0,15.0,0,KHV6OJ08LE0S
1999996,2,25,1,759.0,858.0,AA,1109,N431AA,36.0,DFW,AUS,190,6.0,17.0,0,WM56NFH3B82U
1999997,9,23,2,1226.0,2109.0,DL,162,N3763D,293.0,SEA,JFK,2421,30.0,20.0,0,K6UIWFZ0R1T7
1999998,7,26,6,729.0,929.0,US,1099,N956UW,97.0,PHL,JAX,742,3.0,20.0,0,RMFGZOY59UCK


In [57]:
columns = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "Dest"]

In [58]:
CatBoost = CatBoostClassifier(cat_features = columns, depth = 13)

In [59]:
target = df["target"]

In [60]:
df_new = df_new.fillna(0)

In [61]:
df_new['DepTime'] = df_new['DepTime'] / 100

In [62]:
df_new['ArrTime'] = df_new['ArrTime'] / 100

In [63]:
origin_mte = df[['Origin', 'target']].groupby('Origin')['target'].mean()
origin_mte.name = 'Origin_MTE'
df_new = pd.merge(df_new, origin_mte, how='left', on='Origin')

In [64]:
Dest_mte = df[['Dest', 'target']].groupby('Dest')['target'].mean()
Dest_mte.name = 'Dest_MTE'
df_new = pd.merge(df_new, Dest_mte, how='left', on='Dest')

In [65]:
df_new = df_new.drop("Id", axis = 1)
df_new = df_new.drop("TailNum", axis = 1)

In [66]:
smth = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']
df.insert(loc=len(df.columns), column='Smth', value=smth) 

In [67]:
df_new.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Origin_MTE,Dest_MTE
0,8,14,4,13.34,14.40,XE,430,54.0,FAT,SAN,314,3.0,9.0,0.203920,0.301685
1,5,3,6,17.06,20.16,XE,2567,103.0,BNA,EWR,748,9.0,18.0,0.302108,0.418648
2,7,11,5,13.23,15.07,XE,2989,125.0,CLT,IAH,913,6.0,33.0,0.297510,0.250599
3,12,11,4,20.39,22.38,WN,3743,224.0,BWI,DEN,1491,5.0,10.0,0.314572,0.277956
4,5,23,5,16.28,17.44,UA,926,60.0,LAX,SFO,337,3.0,13.0,0.289365,0.344834


In [68]:
Cat = CatBoost.fit(df_new, target)

Learning rate set to 0.264529
0:	learn: 0.6151986	total: 5.57s	remaining: 1h 32m 48s
1:	learn: 0.5684612	total: 10.7s	remaining: 1h 28m 52s
2:	learn: 0.5437394	total: 16s	remaining: 1h 28m 48s
3:	learn: 0.5287539	total: 20.9s	remaining: 1h 26m 52s
4:	learn: 0.5195960	total: 26.2s	remaining: 1h 26m 48s
5:	learn: 0.5147907	total: 30.5s	remaining: 1h 24m 20s
6:	learn: 0.5073091	total: 36.5s	remaining: 1h 26m 23s
7:	learn: 0.5018397	total: 41.3s	remaining: 1h 25m 22s
8:	learn: 0.4993372	total: 46.7s	remaining: 1h 25m 42s
9:	learn: 0.4974674	total: 51.6s	remaining: 1h 25m 6s
10:	learn: 0.4958056	total: 57s	remaining: 1h 25m 25s
11:	learn: 0.4942454	total: 1m 2s	remaining: 1h 25m 38s
12:	learn: 0.4928587	total: 1m 7s	remaining: 1h 25m 18s
13:	learn: 0.4908091	total: 1m 12s	remaining: 1h 24m 46s
14:	learn: 0.4897553	total: 1m 17s	remaining: 1h 25m 1s
15:	learn: 0.4879064	total: 1m 22s	remaining: 1h 24m 55s
16:	learn: 0.4866513	total: 1m 27s	remaining: 1h 24m 34s
17:	learn: 0.4852408	total: 1m

144:	learn: 0.3676268	total: 13m 19s	remaining: 1h 18m 36s
145:	learn: 0.3669260	total: 13m 26s	remaining: 1h 18m 37s
146:	learn: 0.3658822	total: 13m 32s	remaining: 1h 18m 37s
147:	learn: 0.3650300	total: 13m 39s	remaining: 1h 18m 37s
148:	learn: 0.3640815	total: 13m 44s	remaining: 1h 18m 31s
149:	learn: 0.3638131	total: 13m 52s	remaining: 1h 18m 38s
150:	learn: 0.3633766	total: 13m 59s	remaining: 1h 18m 37s
151:	learn: 0.3628460	total: 14m 5s	remaining: 1h 18m 39s
152:	learn: 0.3624727	total: 14m 11s	remaining: 1h 18m 32s
153:	learn: 0.3618201	total: 14m 17s	remaining: 1h 18m 29s
154:	learn: 0.3613715	total: 14m 24s	remaining: 1h 18m 32s
155:	learn: 0.3607332	total: 14m 30s	remaining: 1h 18m 30s
156:	learn: 0.3601604	total: 14m 36s	remaining: 1h 18m 27s
157:	learn: 0.3598247	total: 14m 42s	remaining: 1h 18m 24s
158:	learn: 0.3591573	total: 14m 48s	remaining: 1h 18m 20s
159:	learn: 0.3585779	total: 14m 55s	remaining: 1h 18m 21s
160:	learn: 0.3581045	total: 15m 1s	remaining: 1h 18m 20s

285:	learn: 0.3038974	total: 28m 47s	remaining: 1h 11m 53s
286:	learn: 0.3036589	total: 28m 55s	remaining: 1h 11m 50s
287:	learn: 0.3034505	total: 29m 2s	remaining: 1h 11m 46s
288:	learn: 0.3029947	total: 29m 7s	remaining: 1h 11m 40s
289:	learn: 0.3025902	total: 29m 14s	remaining: 1h 11m 36s
290:	learn: 0.3022604	total: 29m 22s	remaining: 1h 11m 34s
291:	learn: 0.3018639	total: 29m 29s	remaining: 1h 11m 31s
292:	learn: 0.3016990	total: 29m 37s	remaining: 1h 11m 29s
293:	learn: 0.3014935	total: 29m 43s	remaining: 1h 11m 23s
294:	learn: 0.3012117	total: 29m 50s	remaining: 1h 11m 17s
295:	learn: 0.3009405	total: 29m 58s	remaining: 1h 11m 17s
296:	learn: 0.3006801	total: 30m 6s	remaining: 1h 11m 16s
297:	learn: 0.3001079	total: 30m 13s	remaining: 1h 11m 12s
298:	learn: 0.2996135	total: 30m 20s	remaining: 1h 11m 7s
299:	learn: 0.2993929	total: 30m 26s	remaining: 1h 11m 2s
300:	learn: 0.2990990	total: 30m 33s	remaining: 1h 10m 58s
301:	learn: 0.2987500	total: 30m 40s	remaining: 1h 10m 52s
30

428:	learn: 0.2641683	total: 43m 38s	remaining: 58m 4s
429:	learn: 0.2641104	total: 43m 43s	remaining: 57m 57s
430:	learn: 0.2638991	total: 43m 49s	remaining: 57m 51s
431:	learn: 0.2637276	total: 43m 54s	remaining: 57m 44s
432:	learn: 0.2635080	total: 44m 1s	remaining: 57m 38s
433:	learn: 0.2632655	total: 44m 6s	remaining: 57m 31s
434:	learn: 0.2630657	total: 44m 11s	remaining: 57m 23s
435:	learn: 0.2626564	total: 44m 16s	remaining: 57m 16s
436:	learn: 0.2623184	total: 44m 22s	remaining: 57m 9s
437:	learn: 0.2620901	total: 44m 30s	remaining: 57m 6s
438:	learn: 0.2619498	total: 44m 40s	remaining: 57m 5s
439:	learn: 0.2617524	total: 44m 47s	remaining: 57m
440:	learn: 0.2615954	total: 44m 53s	remaining: 56m 54s
441:	learn: 0.2613972	total: 45m 1s	remaining: 56m 49s
442:	learn: 0.2612015	total: 45m 7s	remaining: 56m 43s
443:	learn: 0.2609204	total: 45m 12s	remaining: 56m 36s
444:	learn: 0.2606128	total: 45m 21s	remaining: 56m 34s
445:	learn: 0.2605641	total: 45m 27s	remaining: 56m 28s
446:

576:	learn: 0.2302277	total: 58m 50s	remaining: 43m 8s
577:	learn: 0.2299615	total: 58m 56s	remaining: 43m 1s
578:	learn: 0.2296999	total: 59m	remaining: 42m 54s
579:	learn: 0.2294838	total: 59m 5s	remaining: 42m 47s
580:	learn: 0.2293035	total: 59m 11s	remaining: 42m 41s
581:	learn: 0.2291113	total: 59m 16s	remaining: 42m 34s
582:	learn: 0.2289948	total: 59m 22s	remaining: 42m 28s
583:	learn: 0.2288476	total: 59m 28s	remaining: 42m 22s
584:	learn: 0.2286965	total: 59m 34s	remaining: 42m 15s
585:	learn: 0.2285503	total: 59m 40s	remaining: 42m 9s
586:	learn: 0.2284015	total: 59m 46s	remaining: 42m 3s
587:	learn: 0.2282774	total: 59m 52s	remaining: 41m 57s
588:	learn: 0.2281756	total: 59m 58s	remaining: 41m 51s
589:	learn: 0.2279362	total: 1h 3s	remaining: 41m 44s
590:	learn: 0.2277550	total: 1h 9s	remaining: 41m 38s
591:	learn: 0.2277104	total: 1h 14s	remaining: 41m 31s
592:	learn: 0.2276171	total: 1h 19s	remaining: 41m 24s
593:	learn: 0.2273734	total: 1h 25s	remaining: 41m 17s
594:	lea

719:	learn: 0.2045025	total: 1h 13m 4s	remaining: 28m 25s
720:	learn: 0.2044002	total: 1h 13m 10s	remaining: 28m 18s
721:	learn: 0.2043161	total: 1h 13m 16s	remaining: 28m 12s
722:	learn: 0.2041714	total: 1h 13m 20s	remaining: 28m 6s
723:	learn: 0.2040699	total: 1h 13m 26s	remaining: 27m 59s
724:	learn: 0.2039990	total: 1h 13m 31s	remaining: 27m 53s
725:	learn: 0.2039837	total: 1h 13m 37s	remaining: 27m 47s
726:	learn: 0.2037355	total: 1h 13m 43s	remaining: 27m 40s
727:	learn: 0.2034974	total: 1h 13m 48s	remaining: 27m 34s
728:	learn: 0.2033427	total: 1h 13m 53s	remaining: 27m 28s
729:	learn: 0.2031381	total: 1h 13m 59s	remaining: 27m 22s
730:	learn: 0.2030207	total: 1h 14m 4s	remaining: 27m 15s
731:	learn: 0.2027399	total: 1h 14m 12s	remaining: 27m 10s
732:	learn: 0.2025209	total: 1h 14m 17s	remaining: 27m 3s
733:	learn: 0.2023570	total: 1h 14m 23s	remaining: 26m 57s
734:	learn: 0.2021479	total: 1h 14m 29s	remaining: 26m 51s
735:	learn: 0.2019999	total: 1h 14m 35s	remaining: 26m 45s
7

859:	learn: 0.1850488	total: 1h 26m 47s	remaining: 14m 7s
860:	learn: 0.1848899	total: 1h 26m 53s	remaining: 14m 1s
861:	learn: 0.1847081	total: 1h 26m 59s	remaining: 13m 55s
862:	learn: 0.1845810	total: 1h 27m 4s	remaining: 13m 49s
863:	learn: 0.1844674	total: 1h 27m 11s	remaining: 13m 43s
864:	learn: 0.1843639	total: 1h 27m 18s	remaining: 13m 37s
865:	learn: 0.1842633	total: 1h 27m 25s	remaining: 13m 31s
866:	learn: 0.1841905	total: 1h 27m 30s	remaining: 13m 25s
867:	learn: 0.1841110	total: 1h 27m 35s	remaining: 13m 19s
868:	learn: 0.1839977	total: 1h 27m 41s	remaining: 13m 13s
869:	learn: 0.1839356	total: 1h 27m 47s	remaining: 13m 7s
870:	learn: 0.1837764	total: 1h 27m 52s	remaining: 13m
871:	learn: 0.1836211	total: 1h 27m 57s	remaining: 12m 54s
872:	learn: 0.1834439	total: 1h 28m 3s	remaining: 12m 48s
873:	learn: 0.1832238	total: 1h 28m 9s	remaining: 12m 42s
874:	learn: 0.1831612	total: 1h 28m 13s	remaining: 12m 36s
875:	learn: 0.1830506	total: 1h 28m 19s	remaining: 12m 30s
876:	le

In [69]:
test = test.fillna(0)

In [70]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Id
0,8,15,5,806.0,957.0,UA,1157,N834UA,111.0,DFW,DEN,641,8.0,52.0,QKXNUJO50EPF
1,10,14,2,2020.0,2133.0,WN,1965,N203WN,49.0,PHX,ONT,325,4.0,20.0,Y2XFDOHQK6VM
2,6,1,7,2245.0,9.0,FL,910,N943AT,67.0,ATL,PHF,508,6.0,11.0,TU5QMACJHR10
3,12,31,3,1638.0,1753.0,OO,5732,N567SW,66.0,GEG,SEA,224,3.0,6.0,0731ET5Z9FWJ
4,10,23,4,1544.0,2358.0,AA,22,N329AA,289.0,LAX,JFK,2475,12.0,13.0,5BNGICLQFKSO


In [71]:
test = test.drop("TailNum", axis = 1)

In [72]:
test = test.fillna(0)

In [73]:
test['DepTime'] = test['DepTime'] / 100

In [74]:
test['ArrTime'] = test['ArrTime'] / 100

In [75]:
test = test.drop("Id", axis = 1)

In [76]:
df["Smth2"] = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']

In [77]:
test["Origin_MTE"] = df_new["Origin_MTE"]
test["Dest_MTE"] = df_new["Dest_MTE"]

In [78]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Origin_MTE,Dest_MTE
0,8,15,5,8.06,9.57,UA,1157,111.0,DFW,DEN,641,8.0,52.0,0.203920,0.301685
1,10,14,2,20.20,21.33,WN,1965,49.0,PHX,ONT,325,4.0,20.0,0.302108,0.418648
2,6,1,7,22.45,0.09,FL,910,67.0,ATL,PHF,508,6.0,11.0,0.297510,0.250599
3,12,31,3,16.38,17.53,OO,5732,66.0,GEG,SEA,224,3.0,6.0,0.314572,0.277956
4,10,23,4,15.44,23.58,AA,22,289.0,LAX,JFK,2475,12.0,13.0,0.289365,0.344834


In [79]:
proba = CatBoost.predict_proba(test)[:, 1]

In [80]:
submission = pd.read_csv("sample_submission.csv")

In [81]:
submission['target'] = proba

In [82]:
submission.to_csv("submission.csv", index = False, header = True)